In [3]:
pip install datasets


In [4]:
!pip install huggingface_hub datasets


In [5]:
from huggingface_hub import hf_hub_download
from huggingface_hub import login

login()

import json



In [6]:
!git clone https://huggingface.co/datasets/kdave/Indian_Financial_News


Cloning into 'Indian_Financial_News'...
remote: Enumerating objects: 10, done.
remote: Total 10 (delta 0), reused 0 (delta 0), pack-reused 10 (from 1)
Unpacking objects: 100% (10/10), 3.06 KiB | 1.02 MiB/s, done.


In [7]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("Indian_Financial_News/training_data_26000.csv")
print(df.columns)


Index(['URL', 'Content', 'Summary', 'Sentiment'], dtype='object')


In [8]:
ds = Dataset.from_pandas(df)
print(ds[0])


{'URL': 'https://www.moneycontrol.com/news/business/economy/covid-19-pandemic-us-consumer-spending-tumble-in-april-5334861.html', 'Content': 'US consumer spending dropped by a record in April as the COVID-19 pandemic undercut demand, buttressing expectations that the economy could contract in the second quarter at its steepest pace since the Great Depression.\n\nThe Commerce Department said on Friday consumer spending, which accounts for more than two-thirds of U.S. economic activity, plunged 13.6 percent last month. That was the biggest drop since the government started tracking series in 1959, and followed a 6.9 percent tumble in March.\n\nEconomists polled by Reuters had forecast consumer spending plummeting 12.6 percent in April.\n\nFollow our full coverage of the coronavirus pandemic here.', 'Summary': 'consumer spending plunges 13.6 percent in April. that was the biggest drop since the government started tracking series in 1959. consumer spending accounts for more than two-thirds

In [9]:
print("Number of records:", len(ds))
print("Columns:", ds.column_names)


Number of records: 26961
Columns: ['URL', 'Content', 'Summary', 'Sentiment']


In [10]:
summary_lengths = [len(x.split()) for x in ds['Summary'] if isinstance(x, str)]
content_lengths = [len(x.split()) for x in ds['Content'] if isinstance(x, str)]

import numpy as np

print("\nSummary Stats:")
print("  Avg length:", np.mean(summary_lengths))
print("  Min length:", np.min(summary_lengths))
print("  Max length:", np.max(summary_lengths))

print("\nContent Stats:")
print("  Avg length:", np.mean(content_lengths))
print("  Min length:", np.min(content_lengths))
print("  Max length:", np.max(content_lengths))



Summary Stats:
  Avg length: 61.362115648529354
  Min length: 1
  Max length: 108

Content Stats:
  Avg length: 609.7049070880161
  Min length: 9
  Max length: 8328


In [11]:
def is_content_valid(example):
    return isinstance(example["Content"], str) and len(example["Content"].split()) >= 100

def is_summary_valid(example):
    return isinstance(example["Summary"], str) and len(example["Summary"].split()) >= 10

a_filtered_ds = ds.filter(is_content_valid)

b_filtered_ds = a_filtered_ds.filter(is_summary_valid)
print("Filtered dataset size:", len(b_filtered_ds))


Filter:   0%|          | 0/26961 [00:00<?, ? examples/s]

Filter:   0%|          | 0/26254 [00:00<?, ? examples/s]

Filtered dataset size: 26233


In [12]:
summary_lengths = [len(x.split()) for x in b_filtered_ds['Summary'] if isinstance(x, str)]
content_lengths = [len(x.split()) for x in b_filtered_ds['Content'] if isinstance(x, str)]

import numpy as np

print("\nSummary Stats (Filtered for extreme short entry):")
print("  Avg length:", np.mean(summary_lengths))
print("  Min length:", np.min(summary_lengths))
print("  Max length:", np.max(summary_lengths))

print("\nContent Stats: (Filtered for extreme short entry)")
print("  Avg length:", np.mean(content_lengths))
print("  Min length:", np.min(content_lengths))
print("  Max length:", np.max(content_lengths))



Summary Stats (Filtered for extreme short entry):
  Avg length: 61.58052834216445
  Min length: 13
  Max length: 98

Content Stats: (Filtered for extreme short entry)
  Avg length: 624.5348225517478
  Min length: 100
  Max length: 8328


In [13]:
print(b_filtered_ds[1])

{'URL': 'https://www.businesstoday.in/top-story/state-run-banks-need-urgent-capital-of-rs-1-trillion-due-to-weak-market-valuations-crisil/story/288921.html', 'Content': 'State-run lenders require an urgent Rs 1.2 trillion in the capital in the next five months and government will have to take a bulk of the tab due to the weak market valuations of these NPA-saddled banks, says report.\n\nThis is a little more than double the budgeted Rs 53,000-crore of capital infusion for the current fiscal year, Crisil senior director Krishnan Sitaraman said in a report Tuesday.\n\nIf the government decides to meet this need, this will put further pressure on the fiscal maths, thus its ability to meet the 3.3 per cent fiscal deficit target for the current fiscal year. Already the government has used up over 95 per cent of the deficit target or the market borrowings as of October end.\n\nThe report comes even as the government is asking the Reserve Bank to lower the minimum capital requirements by gett

**now summary and content has been filtered by length**
b_filtered_ds

In [14]:
import re
import nltk

nltk.download('punkt')  # only once

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
def clean_text(text):
    # Lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove extra whitespace and newlines
    text = re.sub(r'\s+', ' ', text).strip()

    # Replace unicode smart quotes and dashes with ASCII
    replacements = {
        '’': "'",
        '‘': "'",
        '“': '"',
        '”': '"',
        '–': '-',
        '—': '-',
        '\u2013': '-',  # en dash
        '\u2014': '-',  # em dash
    }
    for k, v in replacements.items():
        text = text.replace(k, v)

    # Remove any remaining non-printable characters
    text = ''.join(c for c in text if c.isprintable())

    return text

def filter_entry(entry):
    # For example, skip if content or summary too short
    if len(entry['Content'].split()) < 30:
        return False
    if len(entry['Summary'].split()) < 5:
        return False
    return True

In [16]:

def clean_example(example):
    example['Content'] = clean_text(example['Content'])
    example['Summary'] = clean_text(example['Summary'])
    return example

# Assuming b_filtered_ds is your Dataset object:
b_filtered_ds = b_filtered_ds.map(clean_example)


Map:   0%|          | 0/26233 [00:00<?, ? examples/s]

In [17]:
print(b_filtered_ds[1])

{'URL': 'https://www.businesstoday.in/top-story/state-run-banks-need-urgent-capital-of-rs-1-trillion-due-to-weak-market-valuations-crisil/story/288921.html', 'Content': 'state-run lenders require an urgent rs 1.2 trillion in the capital in the next five months and government will have to take a bulk of the tab due to the weak market valuations of these npa-saddled banks, says report. this is a little more than double the budgeted rs 53,000-crore of capital infusion for the current fiscal year, crisil senior director krishnan sitaraman said in a report tuesday. if the government decides to meet this need, this will put further pressure on the fiscal maths, thus its ability to meet the 3.3 per cent fiscal deficit target for the current fiscal year. already the government has used up over 95 per cent of the deficit target or the market borrowings as of october end. the report comes even as the government is asking the reserve bank to lower the minimum capital requirements by getting it at

In [18]:
b_filtered_ds.to_csv("cleaned_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/27 [00:00<?, ?ba/s]

113592985

In [19]:
# saving files
df = b_filtered_ds.to_pandas()
df.to_json("cleaned_dataset.jsonl", orient="records", lines=True, index=False)


**DATASET PRE-PROCESSED, NOW BASELINE WIHT TEXTRANK**

In [20]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

def split_sentences(text):
    return nltk.sent_tokenize(text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [21]:
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def textrank(sentences, top_n=5):
    # vectorize sentences with TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # compute similarity matrix
    sim_matrix = cosine_similarity(tfidf_matrix)

    # build graph and apply PageRank
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)

    # rank sentences and pick top N
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    selected = [s for _, s in ranked_sentences[:top_n]]

    return ' '.join(selected)


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
# suppose dataset is a list of dicts or a pandas DataFrame
entry = b_filtered_ds[1]  # or dataset.iloc[1] if pandas

text = entry['Content']  # get the content field

# assuming you have your baseline pipeline functions from before:


sentences = split_sentences(text)
summary = textrank(sentences, top_n=5)

print("Original Content:\n", text)
print("Summary:\n", summary)

Original Content:
 state-run lenders require an urgent rs 1.2 trillion in the capital in the next five months and government will have to take a bulk of the tab due to the weak market valuations of these npa-saddled banks, says report. this is a little more than double the budgeted rs 53,000-crore of capital infusion for the current fiscal year, crisil senior director krishnan sitaraman said in a report tuesday. if the government decides to meet this need, this will put further pressure on the fiscal maths, thus its ability to meet the 3.3 per cent fiscal deficit target for the current fiscal year. already the government has used up over 95 per cent of the deficit target or the market borrowings as of october end. the report comes even as the government is asking the reserve bank to lower the minimum capital requirements by getting it at par with global practices-something the central bank is uncomfortable to meet. it has also reported having turned down the finance ministry demand to 

In [27]:
def baseline_summary_pipeline(text):
  sentences = split_sentences(text)
  summary = textrank(sentences, top_n=5)
  return summary

In [28]:
pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b156d7507cb597f5aad98736fbdae9937a78e08d5510251ffd297204374338fc
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [29]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for entry in b_filtered_ds.select(range(20)):
    generated = baseline_summary_pipeline(entry['Content'])  # your generated summary
    reference = entry['Summary']  # human reference summary

    score = scorer.score(reference, generated)

    rouge1_scores.append(score['rouge1'].fmeasure)
    rouge2_scores.append(score['rouge2'].fmeasure)
    rougeL_scores.append(score['rougeL'].fmeasure)

print("Average ROUGE-1 F1:", sum(rouge1_scores)/len(rouge1_scores))
print("Average ROUGE-2 F1:", sum(rouge2_scores)/len(rouge2_scores))
print("Average ROUGE-L F1:", sum(rougeL_scores)/len(rougeL_scores))


Average ROUGE-1 F1: 0.3117687753680703
Average ROUGE-2 F1: 0.18014261244658297
Average ROUGE-L F1: 0.22399678411130425


In [30]:
len(b_filtered_ds)

26233